In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
from configs import TIMEZONE, LOG_FILE_NAME, set_logger
from datetime import datetime
import pytz
import pandas as pd
import numpy as np
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import GetAssetsRequest
from alpaca.data.historical import StockHistoricalDataClient, CryptoHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest, StockBarsRequest, CryptoLatestQuoteRequest
from alpaca.data.requests import CryptoBarsRequest
from alpaca.trading.models import Order
from alpaca.data.timeframe import TimeFrame
from my_secrets import ALPACA_API_BASE_URL, PAPER_API_ID, PAPER_SECRET_KEY
import logging
set_logger()

In [3]:
from Trade_Class import Stock_Trader, Crypto_Trader
from Market_Monitor import Market_Monitor

### Instantiate `Stock_Trader` and `Crypto Trader` classes. Both of these inherit from `Trader`

In [4]:
stock_trader = Stock_Trader(PAPER_API_ID, PAPER_SECRET_KEY, paper=True)
crypto_trader = Crypto_Trader(PAPER_API_ID, PAPER_SECRET_KEY, paper=True)

Instance of Trader() class created.
Instance of Trader() class created.
Instance of Trader() class created.
Instace of Stock_Trader() class created.
Instace of Stock_Trader() class created.
Instace of Stock_Trader() class created.
Starting new HTTPS connection (1): paper-api.alpaca.markets:443
Starting new HTTPS connection (1): paper-api.alpaca.markets:443
Starting new HTTPS connection (1): paper-api.alpaca.markets:443
https://paper-api.alpaca.markets:443 "GET /v2/assets?asset_class=us_equity HTTP/1.1" 200 None
https://paper-api.alpaca.markets:443 "GET /v2/assets?asset_class=us_equity HTTP/1.1" 200 None
https://paper-api.alpaca.markets:443 "GET /v2/assets?asset_class=us_equity HTTP/1.1" 200 None
Instance of Trader() class created.
Instance of Trader() class created.
Instance of Trader() class created.
Instace of Crypto_Trader() class created.
Instace of Crypto_Trader() class created.
Instace of Crypto_Trader() class created.
Starting new HTTPS connection (1): paper-api.alpaca.markets:4

### Basic `.buy()` operation using a market order

In [34]:
ret = crypto_trader.buy('BTC/USD', price=10000)
print(ret) #return of 0 means the function executed correctly, return of anything else means it did not

ret2 = stock_trader.buy('AAPL', quantity=1)
print(ret2)

0
0


### Checking assets in your portfolio using `.get_all_positions()`, `.get_buying_power()` and `.get_all_orders()`.

In [41]:
stock_trader.get_buying_power()
crypto_trader.get_buying_power()
stock_trader.buying_power, crypto_trader.buying_power

(122979.70068115926, 122979.70068115926)

In [42]:
stock_trader.get_all_positions()  #stock and crypto trader are exactly the same functionality. It will get all 
stock_trader.positions_df.head()  #positions regardless of asset class. You can filter by asset class using the asset_class feature in the df

,asset_id,symbol,exchange,asset_class,avg_entry_price,qty,side,market_value,cost_basis,unrealized_pl,unrealized_plpc,unrealized_intraday_pl,unrealized_intraday_plpc,current_price,lastday_price,change_today
0,b0b6dd9d-8b9b-48a9-ba46-b9d54906e415,AAPL,AssetExchange.NASDAQ,AssetClass.US_EQUITY,153.656667,3.000000,PositionSide.LONG,457.200000,460.970000,-3.770000,-0.008178,-3.770000,-0.008178,152.40,154.65,-0.014549
1,64bbff51-59d6-4b3c-9351-13ad85e3c752,BTCUSD,AssetExchange.FTXU,AssetClass.CRYPTO,23278.951120,1.634041,PositionSide.LONG,37414.317379,38038.751488,-624.434109,-0.016416,-587.225174,-0.015453,22896.81,23256.18,-0.015453


In [52]:
ret = stock_trader.get_all_orders() #So I think if it's in this table that means that the order hasn't gone through yet. This table and the positions_df should be mutually exclusive
if ret==0:
    display(stock_trader.orders_df.head())

,id,client_order_id,created_at,updated_at,submitted_at,filled_at,expired_at,canceled_at,failed_at,replaced_at,...,side,time_in_force,limit_price,stop_price,status,extended_hours,legs,trail_percent,trail_price,hwm
0,9481ce3d-7000-4844-9d64-b0b1f6e42cae,c016d438-01c0-4a24-a3c1-2ea247a484a8,1675909892230821000,1675909892230821000,1675909892230207000,NaN,NaN,NaN,NaN,NaN,...,OrderSide.SELL,TimeInForce.GTC,NaN,NaN,OrderStatus.ACCEPTED,False,NaN,NaN,NaN,NaN


In [53]:
for col in stock_trader.orders_df.columns:
    print("="*80)
    print(col)
    print("="*80)
    print(stock_trader.orders_df[col].unique())
    print()
    print()

id
[UUID('9481ce3d-7000-4844-9d64-b0b1f6e42cae')]


client_order_id
['c016d438-01c0-4a24-a3c1-2ea247a484a8']


created_at
[1675909892230821000]


updated_at
[1675909892230821000]


submitted_at
[1675909892230207000]


filled_at
[nan]


expired_at
[nan]


canceled_at
[nan]


failed_at
[nan]


replaced_at
[nan]


replaced_by
[nan]


replaces
[nan]


asset_id
[UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415')]


symbol
['AAPL']


asset_class
[<AssetClass.US_EQUITY: 'us_equity'>]


notional
[nan]


qty
[1]


filled_qty
[0]


filled_avg_price
[nan]


order_class
[<OrderClass.SIMPLE: 'simple'>]


order_type
[<OrderType.MARKET: 'market'>]


type
[<OrderType.MARKET: 'market'>]


side
[<OrderSide.SELL: 'sell'>]


time_in_force
[<TimeInForce.GTC: 'gtc'>]


limit_price
[nan]


stop_price
[nan]


status
[<OrderStatus.ACCEPTED: 'accepted'>]


extended_hours
[False]


legs
[nan]


trail_percent
[nan]


trail_price
[nan]


hwm
[nan]




### Selling positions using `.sell_all()` and `.sell()`

In [8]:
stock_trader.sell_all()

In [47]:
stock_trader.sell('AAPL', quantity=1)

0

### Cancel all open orders using `.cancel_all_orders()`.

In [56]:
cancel_statuses = stock_trader.cancel_all_orders()
cancel_statuses #This is an API response type of thing. 200 means successfully cancelled

### Get market data using `.get_available_symbols()`, `.get_ask_price()`, `.get_bid_price()`, and `.get_bars()`.

In [59]:
crypto_trader.get_available_symbols()
display(crypto_trader.asset_symbols[:5])

stock_trader.get_available_symbols()
display(stock_trader.asset_symbols[:5]) #Aparently not all of these currently are tradeable
# the call is the same for both stock and crypto however they are actually calling two different methods.

['WBTC/USD', 'MKR/USD', 'BCH/BTC', 'YFI/USDT', 'ALGO/USD']

['FLCX', 'OVLC', 'FKSHF', 'FLDYY', 'FLES']

In [60]:
crypto_trader.get_ask_price('BTC/USD'), crypto_trader.get_bid_price('BTC/USD')

(22874.32, 22871.01)

In [62]:
stock_trader.get_ask_price('AAPL'), stock_trader.get_bid_price('AAPL') #if the stock is not tradeable after hours ask and bid will both be 0.0

(0.0, 0.0)

In [67]:
start = datetime(year=2023, month=1, day=1, hour=0, minute=0, second=0)
end = datetime(year=2023, month=2, day=1, hour=0, minute=0, second=0)
bars_df = crypto_trader.get_bars('BTC/USD', start=start, end=end, time_resolution='day')
display(bars_df.head())

open      high       low     close  \
symbol  timestamp                                                           
BTC/USD 2023-01-01 06:00:00+00:00  16525.00  16695.31  16495.06  16642.78   
        2023-01-02 06:00:00+00:00  16644.37  16779.00  16630.47  16718.82   
        2023-01-03 06:00:00+00:00  16721.01  16900.00  16600.00  16850.59   
        2023-01-04 06:00:00+00:00  16852.62  16984.54  16760.57  16840.50   
        2023-01-05 06:00:00+00:00  16838.47  16873.28  16755.45  16803.47   

                                        volume  trade_count          vwap  
symbol  timestamp                                                          
BTC/USD 2023-01-01 06:00:00+00:00  1651.545517      34598.0  16578.560393  
        2023-01-02 06:00:00+00:00  2285.641776      41442.0  16708.451895  
        2023-01-03 06:00:00+00:00  3790.481395      64870.0  16722.315217  
        2023-01-04 06:00:00+00:00  4847.927197      81921.0  16848.857792  
        2023-01-05 06:00:00+00:00  2580.394993      57458.0  16822.708092

In [72]:
bars_df.index[:5] #as you can see the resulting dataframe has a multi index with the most exterior index as the symbol

MultiIndex([('BTC/USD', '2023-01-01 06:00:00+00:00'),
            ('BTC/USD', '2023-01-02 06:00:00+00:00'),
            ('BTC/USD', '2023-01-03 06:00:00+00:00'),
            ('BTC/USD', '2023-01-04 06:00:00+00:00'),
            ('BTC/USD', '2023-01-05 06:00:00+00:00')],
           names=['symbol', 'timestamp'])

In [73]:
start = datetime(year=2023, month=1, day=1, hour=0, minute=0, second=0)
end = datetime(year=2023, month=2, day=1, hour=0, minute=0, second=0)
bars_df = stock_trader.get_bars('AAPL', start=start, end=end, time_resolution='day')
display(bars_df.head()) # same operation for crypto and stock

open      high     low   close  \
symbol timestamp                                                      
AAPL   2023-01-03 05:00:00+00:00  130.280  130.9000  124.17  125.07   
       2023-01-04 05:00:00+00:00  126.890  128.6557  125.08  126.36   
       2023-01-05 05:00:00+00:00  127.130  127.7400  124.76  125.02   
       2023-01-06 05:00:00+00:00  126.010  130.2900  124.89  129.62   
       2023-01-09 05:00:00+00:00  130.465  133.4100  129.89  130.15   

                                       volume  trade_count        vwap  
symbol timestamp                                                        
AAPL   2023-01-03 05:00:00+00:00  112084324.0    1020782.0  125.725146  
       2023-01-04 05:00:00+00:00   89072202.0     769281.0  126.646333  
       2023-01-05 05:00:00+00:00   81246605.0     663745.0  126.088261  
       2023-01-06 05:00:00+00:00   87758343.0     711520.0  128.198067  
       2023-01-09 05:00:00+00:00   70832518.0     647372.0  131.628178